In [1]:
import time
from IPython.display import display, Javascript

def stay_awake():
    code = """
    function ClickConnect(){
        console.log("🚀 Clicked on connect to keep the notebook awake");
        document.querySelector("colab-connect-button").click()
    }
    setInterval(ClickConnect, 60000)
    """
    display(Javascript(code))

stay_awake()


<IPython.core.display.Javascript object>

In [2]:
!pip install -q transformers datasets scikit-learn sentence-transformers gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 118.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Dataset**

In [3]:
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 🧠 TEXT CLASSIFICATION — MedQuAD
medquad = load_dataset("lavita/MedQuAD")
questions = [item["question"] for item in medquad["train"]]
labels = [item.get("question_type", "unknown") for item in medquad["train"]]

# Simplify 30+ classes into 5
label_mapping = {
    'symptoms': 'Symptom', 'complications': 'Symptom', 'side effects': 'Symptom', 'severe reaction': 'Symptom',
    'causes': 'Diagnosis', 'exams and tests': 'Diagnosis', 'stages': 'Diagnosis', 'genetic changes': 'Diagnosis',
    'susceptibility': 'Diagnosis', 'inheritance': 'Diagnosis', 'how does it work': 'Diagnosis',
    'treatment': 'Treatment', 'prevention': 'Treatment', 'dietary': 'Treatment', 'how effective is it': 'Treatment',
    'research': 'Treatment', 'support groups': 'Treatment',
    'usage': 'Medication', 'indication': 'Medication', 'dose': 'Medication', 'frequency': 'Medication',
    'interactions with medications': 'Medication', 'interactions with herbs and supplements': 'Medication',
    'interactions with foods': 'Medication', 'brand names': 'Medication', 'brand names of combination products': 'Medication',
    'forget a dose': 'Medication', 'storage and disposal': 'Medication', 'contraindication': 'Medication',
    'emergency or overdose': 'Urgent', 'when to contact a medical professional': 'Urgent', 'important warning': 'Urgent',
    'information': 'Other', 'other information': 'Other', 'considerations': 'Other',
    'why get vaccinated': 'Other', 'how can i learn more': 'Other'
}
simplified_labels = [label_mapping.get(label.lower(), 'Other') for label in labels]

# Encode simplified labels
le = LabelEncoder()
encoded_labels = le.fit_transform(simplified_labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(questions, encoded_labels, test_size=0.2, random_state=42)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

(…)-00000-of-00001-e36383d177026d53.parquet:   0%|          | 0.00/10.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/47441 [00:00<?, ? examples/s]

In [4]:
# Step 1: Imports and dataset
from datasets import load_dataset
raw_dataset = load_dataset("ibm-research/MedMentions-ZS", split="train[:1000]")

# Step 2: Create label2id and id2label
ner_labels = list(set(tag for sample in raw_dataset["ner_tags"] for tag in sample))
label2id = {label: i for i, label in enumerate(sorted(ner_labels))}
id2label = {i: label for label, i in label2id.items()}

dataset_infos.json:   0%|          | 0.00/959 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.23M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/109k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/26770 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1289 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1048 [00:00<?, ? examples/s]

**Code For Model Loading for text**

In [5]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

clf_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
clf_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(le.classes_)
)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**NER**

In [6]:
# Step 3: Load model and tokenizer
from transformers import AutoModelForTokenClassification, AutoTokenizer

model_ckpt = "distilbert-base-uncased"
ner_tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
ner_model = AutoModelForTokenClassification.from_pretrained(model_ckpt, num_labels=len(label2id))

ner_model.config.label2id = label2id
ner_model.config.id2label = id2label


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Fine tuning for text**

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch

# Tokenizer and label encoder
clf_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
le = LabelEncoder()
encoded_labels = le.fit_transform(simplified_labels)

# Tokenization
inputs = clf_tokenizer(questions, padding=True, truncation=True, return_tensors="pt")

class MedicalDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

# Split into train/test
from sklearn.model_selection import train_test_split
train_qs, test_qs, train_labels, test_labels = train_test_split(
    questions, encoded_labels, test_size=0.2, stratify=encoded_labels, random_state=42
)

train_inputs = clf_tokenizer(train_qs, padding=True, truncation=True, return_tensors="pt")
test_inputs = clf_tokenizer(test_qs, padding=True, truncation=True, return_tensors="pt")

train_ds = MedicalDataset(train_inputs, train_labels)
test_ds = MedicalDataset(test_inputs, test_labels)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=16)

# Model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(le.classes_)
).to("cuda" if torch.cuda.is_available() else "cpu")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Fine Tuning for NER**

In [8]:
# Step 4: Tokenization
from sklearn.model_selection import train_test_split

def tokenize_and_align_labels(batch):
    tokenized = ner_tokenizer(
        batch["tokens"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True
    )
    labels = []
    for i, word_ids in enumerate(tokenized.word_ids(batch_index=i) for i in range(len(batch["tokens"]))):
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                tag = batch["ner_tags"][i][word_idx]
                label_ids.append(label2id.get(tag, -100))
        labels.append(label_ids)
    tokenized["labels"] = labels
    return tokenized

split_dataset = raw_dataset.train_test_split(test_size=0.2)
tokenized = split_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [9]:
# Step 5: Training setup
from transformers import DataCollatorForTokenClassification, Trainer, TrainingArguments

data_collator = DataCollatorForTokenClassification(tokenizer=ner_tokenizer)

training_args = TrainingArguments(
    output_dir="./ner_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    seed=42,
    report_to="none"
)

trainer = Trainer(
    model=ner_model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=ner_tokenizer,
    data_collator=data_collator
)


<ipython-input-9-9b0dc6944734>:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [10]:
# Step 6: Train
trainer.train()

Step,Training Loss
10,2.543000
20,1.679900
30,1.408600
40,1.300700
50,1.310700
60,1.160500
70,1.230600
80,1.108100
90,1.184600
100,1.010000


TrainOutput(global_step=150, training_loss=1.2757360394795736, metrics={'train_runtime': 30.3333, 'train_samples_per_second': 79.121, 'train_steps_per_second': 4.945, 'total_flos': 78421625856000.0, 'train_loss': 1.2757360394795736, 'epoch': 3.0})

**Text **


In [11]:
from datasets import Dataset

train_dataset = Dataset.from_dict({"text": X_train, "label": y_train})
test_dataset = Dataset.from_dict({"text": X_test, "label": y_test})

text_dataset = {"train": train_dataset, "test": test_dataset}


In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

text_model_name = "distilbert-base-uncased"
text_tokenizer = AutoTokenizer.from_pretrained(text_model_name)

# Set up label mappings
text_label2id = {label: i for i, label in enumerate(le.classes_)}
text_id2label = {i: label for label, i in text_label2id.items()}

text_model = AutoModelForSequenceClassification.from_pretrained(
    text_model_name,
    num_labels=len(text_label2id),
    id2label=text_id2label,
    label2id=text_label2id
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
def preprocess_text(example):
    return text_tokenizer(example["text"], truncation=True, padding="max_length")

text_dataset = {
    "train": text_dataset["train"].map(preprocess_text, batched=True),
    "test": text_dataset["test"].map(preprocess_text, batched=True)
}


Map:   0%|          | 0/37952 [00:00<?, ? examples/s]

Map:   0%|          | 0/9489 [00:00<?, ? examples/s]

In [14]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }


In [15]:
!pip install -q transformers datasets scikit-learn

In [16]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [17]:
from sklearn.metrics import classification_report

test_loader = DataLoader(test_ds, batch_size=16)
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        out = model(input_ids=ids, attention_mask=mask)
        preds = torch.argmax(out.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=le.classes_))

              precision    recall  f1-score   support

   Diagnosis       0.14      0.29      0.19      1536
  Medication       0.00      0.00      0.00      1545
       Other       0.11      0.01      0.02      2991
     Symptom       0.00      0.00      0.00      1357
   Treatment       0.00      0.00      0.00      1391
      Urgent       0.06      0.50      0.10       669

    accuracy                           0.09      9489
   macro avg       0.05      0.13      0.05      9489
weighted avg       0.06      0.09      0.04      9489



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Hyperparameter For Text**

In [19]:
!pip install -q tqdm
from tqdm import tqdm

learning_rates = [2e-5, 3e-5, 5e-5]
results = []

for lr in learning_rates:
    print(f"\n🔧 Training with learning rate: {lr}")

    # 1. Reload model fresh each time
    model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(le.classes_)).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    # 2. Train
    model.train()
    for epoch in range(2):  # Use 2 epochs to save time
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1} | LR={lr}") # Using tqdm to create a progress bar
        for batch in loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            out = model(**batch)
            loss = out.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

    # 3. Evaluate
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in test_loader:
            ids = batch['input_ids'].to(device)
            mask = batch['attention_mask'].to(device)
            y = batch['labels'].to(device)
            out = model(input_ids=ids, attention_mask=mask)
            pred = torch.argmax(out.logits, dim=1)
            preds.extend(pred.cpu().numpy())
            labels.extend(y.cpu().numpy())

    report = classification_report(labels, preds, output_dict=True)
    f1 = report["weighted avg"]["f1-score"]
    results.append((lr, round(f1, 4)))
    print(f"✅ F1 Score for LR={lr}: {f1:.4f}")


🔧 Training with learning rate: 2e-05


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1 | LR=2e-05: 100%|██████████| 2372/2372 [03:02<00:00, 12.99it/s]
Epoch 2 | LR=2e-05: 100%|██████████| 2372/2372 [03:03<00:00, 12.95it/s]


✅ F1 Score for LR=2e-05: 1.0000

🔧 Training with learning rate: 3e-05


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1 | LR=3e-05: 100%|██████████| 2372/2372 [03:03<00:00, 12.91it/s]
Epoch 2 | LR=3e-05: 100%|██████████| 2372/2372 [03:01<00:00, 13.09it/s]


✅ F1 Score for LR=3e-05: 1.0000

🔧 Training with learning rate: 5e-05


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1 | LR=5e-05: 100%|██████████| 2372/2372 [03:01<00:00, 13.06it/s]
Epoch 2 | LR=5e-05: 100%|██████████| 2372/2372 [03:01<00:00, 13.07it/s]


✅ F1 Score for LR=5e-05: 0.9999


**Hyperparamter for NER**

In [20]:
from transformers import TrainingArguments, Trainer

learning_rates = [2e-5, 3e-5, 5e-5]

for lr in learning_rates:
    print(f"\n🔁 Training NER with LR = {lr}")

    training_args = TrainingArguments(
        output_dir=f"./ner_model_lr{lr}",
        learning_rate=lr,
        per_device_train_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
        save_steps=500,
        report_to="none",
        seed=42
    )

    trainer = Trainer(
        model=ner_model,
        args=training_args,
        train_dataset=tokenized["train"],
        eval_dataset=tokenized["test"],
        tokenizer=ner_tokenizer,
        data_collator=data_collator
    )

    trainer.train()



🔁 Training NER with LR = 2e-05


<ipython-input-20-c9fc0f644d50>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,1.048900
20,1.002500
30,0.921700
40,0.913800
50,0.926600
60,0.796600
70,0.865900
80,0.780300
90,0.869800
100,0.759200



🔁 Training NER with LR = 3e-05


<ipython-input-20-c9fc0f644d50>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,0.725000
20,0.708200
30,0.633800
40,0.644300
50,0.646200
60,0.503700
70,0.554400
80,0.524300
90,0.571000
100,0.508600



🔁 Training NER with LR = 5e-05


<ipython-input-20-c9fc0f644d50>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,0.411100
20,0.434700
30,0.378100
40,0.402100
50,0.382400
60,0.262500
70,0.292000
80,0.294100
90,0.317100
100,0.290000


**Evaluation Model For text**

In [21]:
from sklearn.metrics import classification_report

# Run predictions
model.eval()
preds, actuals = [], []
with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        output = model(**batch)
        logits = output.logits
        pred = torch.argmax(logits, axis=1)
        preds.extend(pred.cpu().numpy())
        actuals.extend(batch['labels'].cpu().numpy())

# Classification report
report = classification_report(actuals, preds, target_names=le.classes_)
print(report)

              precision    recall  f1-score   support

   Diagnosis       1.00      1.00      1.00      1536
  Medication       1.00      1.00      1.00      1545
       Other       1.00      1.00      1.00      2991
     Symptom       1.00      1.00      1.00      1357
   Treatment       1.00      1.00      1.00      1391
      Urgent       1.00      1.00      1.00       669

    accuracy                           1.00      9489
   macro avg       1.00      1.00      1.00      9489
weighted avg       1.00      1.00      1.00      9489



**NER**

In [25]:
!pip install -q evaluate

In [26]:
!pip install -q seqeval # Install the seqeval dependency
from evaluate import load
import numpy as np

metric = load("seqeval")

def compute_ner_metrics(predictions, labels):
    true_predictions = [
        [id2label[p] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return results

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [44]:
from evaluate import load
import numpy as np

metric = load("seqeval")

def compute_ner_metrics(predictions, labels):
    true_predictions = [
        [id2label[p] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(pred, lab) if l != -100]
        for pred, lab in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return results

In [52]:
model.eval()  # Set model to evaluation mode
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)  # Inference step
    logits = outputs.logits  # Extract logits from the model output

# Get predicted labels from logits
pred_labels = torch.argmax(logits, dim=-1).cpu().numpy()  # Convert to labels


**Error for text**

In [48]:
wrong_examples = []

for q, pred, true in zip(test_qs, all_preds, all_labels):
    if pred != true:
        wrong_examples.append((q, le.classes_[true], le.classes_[pred]))

# Show a few
for i, (q, true_label, pred_label) in enumerate(wrong_examples[:5]):
    print(f"❌ Q{i+1}: {q}")
    print(f"   ✅ True: {true_label} | ❌ Pred: {pred_label}\n")

❌ Q1: How to prevent Flat feet ?
   ✅ True: Treatment | ❌ Pred: Urgent

❌ Q2: Who should get Haloperidol and why is it prescribed ?
   ✅ True: Medication | ❌ Pred: Diagnosis

❌ Q3: What is (are) Gamma heavy chain disease ?
   ✅ True: Other | ❌ Pred: Urgent

❌ Q4: What is (are) Tennis elbow surgery ?
   ✅ True: Other | ❌ Pred: Urgent

❌ Q5: What is (are) Syndromic microphthalmia, type 3 ?
   ✅ True: Other | ❌ Pred: Urgent



**Error For NER**

In [69]:
print(split_dataset["test"][0])

{'tokens': ['CONCLUSION', 'This', 'rat', 'model', 'resembles', 'the', 'key', 'features', 'of', 'human', 'NAFLD', '/', 'NASH', 'and', 'provides', 'a', 'simple', 'pragmatic', 'experimental', 'model', 'for', 'elucidating', 'the', 'disease', 'prevention', 'and', 'treatment', '.'], 'ner_tags': ['O', 'O', 'B-T204', 'B-T204', 'O', 'O', 'O', 'O', 'O', 'B-T204', 'B-T038', 'O', 'B-T038', 'O', 'O', 'O', 'O', 'B-T204', 'I-T204', 'I-T204', 'O', 'O', 'O', 'O', 'O', 'O', 'B-T058', 'O']}


In [ ]:
for idx in range(5):
    tokens = raw_dataset["tokens"][idx]
    preds = ner_predictions[idx]
    true_tags = ner_labels[idx]

    mismatches = [(t, p, l) for t, p, l in zip(tokens, preds, true_tags) if p != l]

    if mismatches:
        print(f"🔍 Sample {idx}")
        for tok, pred_tag, true_tag in mismatches:
            print(f"  {tok}: pred={pred_tag}, true={true_tag}")
        print("-----")

In [72]:
test_split = split_dataset["test"]


In [76]:
# Step 4: Tokenization
from sklearn.model_selection import train_test_split

def tokenize_and_align_labels(batch):
    tokenized = ner_tokenizer(
        batch["tokens"],
        truncation=True,
        padding="max_length",
        max_length=128,
        is_split_into_words=True
    )
    labels = []
    for i, word_ids in enumerate(tokenized.word_ids(batch_index=i) for i in range(len(batch["tokens"]))):
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                tag = batch["ner_tags"][i][word_idx]
                label_ids.append(label2id.get(tag, -100))
        labels.append(label_ids)
    tokenized["labels"] = labels
    return tokenized

split_dataset = raw_dataset.train_test_split(test_size=0.2)
# Applying the map function to the train and test splits separately
tokenized_train = split_dataset["train"].map(tokenize_and_align_labels, batched=True)
tokenized_test = split_dataset["test"].map(tokenize_and_align_labels, batched=True)

# Creating a dictionary for tokenized train and test splits
tokenized_dataset = {"train": tokenized_train, "test": tokenized_test}

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [80]:
ner_labels = [sample["ner_tags"] for sample in raw_dataset] # ner_labels was empty before, needs to be populated

for idx in range(min(5, len(raw_dataset))): # to prevent going beyond the available samples
    tokens = raw_dataset["tokens"][idx]
    preds = ner_predictions[idx]
    true_tags = ner_labels[idx]

    mismatches = [(t, p, l) for t, p, l in zip(tokens, preds, true_tags) if p != l]

    if mismatches:
        print(f"🔍 Sample {idx}")
        for tok, pred_tag, true_tag in mismatches:
            print(f"  {tok}: pred={pred_tag}, true={true_tag}")
        print("-----")

🔍 Sample 0
  DCTN4: pred=O, true=B-T103
  a: pred=B-T038, true=O
  modifier: pred=B-T038, true=O
  of: pred=B-T038, true=O
  chronic: pred=O, true=B-T038
  Pseudomonas: pred=O, true=I-T038
  aeruginosa: pred=O, true=I-T038
  infection: pred=O, true=I-T038
  cystic: pred=O, true=B-T038
  fibrosis: pred=B-T103, true=I-T038
-----
🔍 Sample 1
  Pseudomonas: pred=O, true=B-T038
  aeruginosa: pred=B-T103, true=I-T038
  (: pred=I-T103, true=I-T038
  Pa: pred=I-T103, true=I-T038
  ): pred=B-T103, true=I-T038
  infection: pred=B-T103, true=I-T038
  cystic: pred=O, true=B-T038
  fibrosis: pred=O, true=I-T038
  CF: pred=O, true=B-T038
  patients: pred=B-T103, true=O
  is: pred=B-T103, true=O
  worse: pred=B-T038, true=O
  long: pred=B-T038, true=O
  -: pred=I-T038, true=O
  and: pred=B-T103, true=O
  ,: pred=B-T038, true=O
  chronic: pred=O, true=B-T038
  Pa: pred=O, true=I-T038
  infection: pred=B-T103, true=I-T038
  ): pred=B-T038, true=O
  associated: pred=B-T103, true=O
  ,: pred=B-T103, true=

**Inference Pipeline**

In [122]:
def classify_question(text):
    # Tokenize and classify
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    with torch.no_grad():
        output = model(**inputs)
        logits = output.logits
        probs = torch.nn.functional.softmax(logits, dim=1)
        # Get the index of the highest probability, rather than the raw value
        pred = torch.argmax(probs, dim=1)[0].item()  # [0] selects the first element of the tensor
        confidence = torch.max(probs).item()

    label = le.inverse_transform([pred])[0]
    return label, confidence

In [96]:
# Assuming you have already loaded your NER model and tokenized the input

def extract_entities(text):
    ner_model.eval()
    ner_predictions = []
    ner_labels = []

    # Tokenize the input text for NER
    tokenized_input = ner_tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(ner_model.device)
    input_ids = tokenized_input["input_ids"]
    attention_mask = tokenized_input["attention_mask"]

    with torch.no_grad():
        output = ner_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = output.logits
        pred_ids = torch.argmax(logits, dim=2).squeeze().cpu().tolist()

    # Convert predictions to labels
    ner_predictions = [id2label[p] for p in pred_ids]

    return ner_predictions

# Example input
text_ner = "What are the side effects of ibuprofen?"
entities = extract_entities(text_ner)

print(f"Entities: {entities}")


Entities: ['B-T017', 'B-T082', 'B-T082', 'B-T017', 'B-T017', 'B-T017', 'B-T017', 'B-T017', 'B-T062', 'B-T062', 'B-T062', 'B-T017', 'B-T017']


In [97]:
def pipeline(text):
    # Get the text classification (question type) result
    question_type, confidence = classify_question(text)

    # Get NER results (named entities)
    ner_results = extract_entities(text)

    # Return the results in a formatted output
    return f"🩺 **Question Type**: {question_type}\n" \
           f"📊 **Confidence**: {confidence:.2f}\n" \
           f"🧬 **Entities**: {', '.join(ner_results)}"

# Example input
text_input = "What are the side effects of ibuprofen?"
output = pipeline(text_input)

print(output)


🩺 **Question Type**: Diagnosis
📊 **Confidence**: 0.51
🧬 **Entities**: B-T017, B-T082, B-T082, B-T017, B-T017, B-T017, B-T017, B-T017, B-T062, B-T062, B-T062, B-T017, B-T017


In [98]:
def extract_entities(text):
    ner_model.eval()
    ner_predictions = []

    # Tokenize the input text for NER
    tokenized_input = ner_tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(ner_model.device)
    input_ids = tokenized_input["input_ids"]
    attention_mask = tokenized_input["attention_mask"]

    with torch.no_grad():
        output = ner_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = output.logits
        pred_ids = torch.argmax(logits, dim=2).squeeze().cpu().tolist()

    # Get the tokens from the tokenizer (map back to words)
    tokens = ner_tokenizer.convert_ids_to_tokens(input_ids.squeeze().cpu().tolist())

    # Now map the predicted labels (e.g., B-T038) back to the corresponding words
    entities = []
    for token, label_id in zip(tokens, pred_ids):
        label = id2label[label_id]
        if label != "O":  # Ignore non-entity tokens (O stands for "Other")
            entities.append(f"{token} ({label})")

    return entities

# Example input
text_ner = "What are the side effects of ibuprofen?"
entities = extract_entities(text_ner)

print(f"Entities: {entities}")


Entities: ['[CLS] (B-T017)', 'what (B-T082)', 'are (B-T082)', 'the (B-T017)', 'side (B-T017)', 'effects (B-T017)', 'of (B-T017)', 'ib (B-T017)', '##up (B-T062)', '##ro (B-T062)', '##fen (B-T062)', '? (B-T017)', '[SEP] (B-T017)']


In [99]:
def extract_entities(text):
    ner_model.eval()
    ner_predictions = []

    tokenized_text = ner_tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device) # Use ner_tokenizer
    input_ids = tokenized_text["input_ids"].to(device)
    attention_mask = tokenized_text["attention_mask"].to(device)

    with torch.no_grad():
        outputs = ner_model(**tokenized_text)  # Pass the entire tokenized dictionary
        logits = outputs.logits
        pred_ids = torch.argmax(logits, dim=2).squeeze().cpu().tolist()

    # Assuming `id2label` exists for ner_model
    ner_predictions = [ner_model.config.id2label[p] for p in pred_ids if p != -100]

    return ner_predictions

In [88]:
def extract_entities(text):
    ner_model.eval()
    ner_predictions = []

    # Tokenize the input text for NER
    tokenized_input = ner_tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(ner_model.device)
    input_ids = tokenized_input["input_ids"]
    attention_mask = tokenized_input["attention_mask"]

    with torch.no_grad():
        output = ner_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = output.logits
        pred_ids = torch.argmax(logits, dim=2).squeeze().cpu().tolist()

    # Get the tokens from the tokenizer (map back to words)
    tokens = ner_tokenizer.convert_ids_to_tokens(input_ids.squeeze().cpu().tolist())

    # Merge subword tokens (those starting with "##")
    entities = []
    current_entity = ""
    for token, label_id in zip(tokens, pred_ids):
        label = id2label[label_id]
        if label != "O":  # Ignore non-entity tokens (O stands for "Other")
            if token.startswith("##"):  # This is a subword token, append it to the current entity
                current_entity += token[2:]  # Remove the "##" prefix
            else:  # It's a new entity
                if current_entity:
                    entities.append(current_entity)  # Append the completed entity
                current_entity = token  # Start a new entity
    if current_entity:
        entities.append(current_entity)  # Append the last entity

    # Format the entities with their labels
    formatted_entities = [f"{entity} ({label})" for entity, label in zip(entities, pred_ids)]

    return formatted_entities

# Example input
text_ner = "What are the side effects of ibuprofen?"
entities = extract_entities(text_ner)

print(f"Entities: {entities}")


Entities: ['side (22)', 'ibuprofen (22)']


In [102]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification
import gradio as gr

# Load models
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
text_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
ner_model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

# Move models to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
text_model.to(device)
ner_model.to(device)

# Define classification function
def classify_question(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = text_model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()
        confidence = torch.max(probs).item()

    label = text_model.config.id2label[pred]  # Fetch the predicted label
    severity = 2  # Example severity level (you can adjust as needed)
    return label, confidence, severity

# Define NER function
def extract_entities(text):
    ner_model.eval()
    ner_predictions = []

    tokenized_text = tokenizer(text, return_tensors="pt")
    # Remove unsqueeze(0) to avoid adding extra dimension:
    input_ids = tokenized_text["input_ids"].to(device)
    attention_mask = tokenized_text["attention_mask"].to(device)

    with torch.no_grad():
        output = ner_model(input_ids=input_ids, attention_mask=attention_mask)
        logits = output.logits
        pred_ids = torch.argmax(logits, dim=2).squeeze().cpu().tolist()

    # Assuming `id2label` exists
    ner_predictions = [ner_model.config.id2label[p] for p in pred_ids if p != -100]

    return ner_predictions

# Combine both text classification and NER
def pipeline(text):
    classification = classify_question(text)
    entities = extract_entities(text)
    result = (
        f"🩺 **Prediction**: {classification[0]}\n"
        f"📊 **Confidence**: {classification[1]:.2f}\n"
        f"⚠️ **Severity Score**: {classification[2]}\n"
        f"🧬 **Entities**: {', '.join(entities)}"
    )
    return result

# Test manually before running Gradio
text_input = "What are the side effects of ibuprofen?"
output = pipeline(text_input)
print(output)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model f

🩺 **Prediction**: LABEL_0
📊 **Confidence**: 0.54
⚠️ **Severity Score**: 2
🧬 **Entities**: O, I-ORG, I-ORG, O, O, O, O, O, I-PER, I-PER, I-PER, O, O


In [103]:
!pip install gradio


In [146]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.eval()  # Set the model to evaluation mode


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [147]:
import torch

# Severity and label maps
severity_map = {
    "Urgent": 3,
    "Diagnosis": 2,
    "Treatment": 2,
    "Symptom": 1,
    "Medication": 1,
    "Other": 0
}

label_map = {0: "Symptom", 1: "Diagnosis", 2: "Treatment", 3: "Urgent"}

# Dummy NER function (replace with actual NER logic)
def extract_entities(text):
    # Dummy entities, replace with actual NER logic
    return ["headache", "fever"]  # Placeholder for medical entity extraction

# Function to classify the question and extract entities
def classify_question(text):
    # Tokenize and classify the question
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1)
        pred = torch.argmax(probs, dim=1).item()  # Get the index of the highest probability
        confidence = torch.max(probs).item()

    label = label_map.get(pred, "Other")
    severity = severity_map.get(label, 0)

    # Extract NER entities
    ner_results = extract_entities(text)

    # Format the output in markdown
    result = (
        f"🩺 **Prediction**: {label}\n"
        f"📊 **Confidence**: {confidence:.2f}\n"
        f"⚠️ **Severity Score**: {severity} (Higher = More urgent)\n\n"
        f"🧬 **Entities**: {', '.join(ner_results)}"  # Show NER entities
    )

    return result


In [159]:
sample_text = "I am feeling dizzy?"
output = classify_question(sample_text)
print(output)


🩺 **Prediction**: Diagnosis
📊 **Confidence**: 0.54
⚠️ **Severity Score**: 2 (Higher = More urgent)

🧬 **Entities**: headache, fever


In [163]:
import gradio as gr

In [169]:
interface = gr.Interface(
    fn=classify_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask a healthcare question..."),
    outputs=gr.Markdown(),  # Use this for displaying Markdown output
    title="Healthcare Question Classifier + Severity + NER",
    description="Classifies the healthcare question type, displays severity score, and extracts medical entities using NER."
)


In [171]:
# Test with different example
sample_text_2 = "What are the side effects of ibuprofen?"
output_2 = classify_question(sample_text_2)
print(output_2)


🩺 **Prediction**: Diagnosis
📊 **Confidence**: 0.54
⚠️ **Severity Score**: 2 (Higher = More urgent)

🧬 **Entities**: headache, fever


In [173]:
# Test with different example
sample_text_3 = "I am Having pain in my leg"
output_3 = classify_question(sample_text_3)
print(output_3)


🩺 **Prediction**: Diagnosis
📊 **Confidence**: 0.52
⚠️ **Severity Score**: 2 (Higher = More urgent)

🧬 **Entities**: headache, fever
